In [4]:
import pandas as pd
from tiingo import TiingoClient
from ta.momentum import RSIIndicator, stoch
from ta.trend import MACD as md
from ta.volatility import AverageTrueRange as atr

In [243]:
client = TiingoClient({'api_key' : '7fb8e25ac6335f846ae82d3f3a28ed535f313592'})

df = pd.read_csv(R"X:\GitHub\beatingENd\BeatingEnDUS\Own Screener\IBD50.csv")

# datalist = (df['Ticker'].to_list())
datalist = ['NFLX', 'AAPL', 'GOOG']

data = client.get_dataframe(datalist,
                                    metric_name='adjClose',
                                      frequency='daily',
                                      startDate='2015-02-02',
                                      endDate='2020-10-10')

In [269]:
def RSI_base(data,ticker):
    MACD = md(close=data[ticker])
    data["MACD_diff"] = MACD.macd_diff()
    RSI = RSIIndicator(close=data[ticker])
    data["RSI"] = RSI.rsi()
    data["Buy_Signal"] = 0
    data["Pos"] = 0
    data["Sell_Signal"] = 0
    data["SL"] = 0 
    data["TP"] = 0
    data["TW"] = 0.0

    data = data.dropna()

    for i in range(0,len(data)):
        if data["Pos"][i-1] == 0:
            if data["RSI"].iloc[i] > 50 and data["MACD_diff"][i] > 0 and (data["Pos"][i] == 0):
                data["Buy_Signal"][i] = 1
                data["Pos"][i] = 1
                data["SL"][i] = (data[ticker][i]*0.97)
                data["TP"][i] = (data[ticker][i]*1.5)
            elif data["RSI"].iloc[i] < 50 and data["MACD_diff"][i] < 0 and (data["Pos"][i] == 0): 
                data["Buy_Signal"][i] = 1
                data["Pos"][i] = -1
                data["SL"][i] = (data[ticker][i]*1.03)
                data["TP"][i] = (data[ticker][i]*0.95)
            else:
                data["Buy_Signal"][i] = 0 
                data["Pos"][i] = 0

        elif data["Pos"][i-1] == 1:
            data["SL"][i] = data["SL"][i-1] 
            data["TP"][i] = data["TP"][i-1]
            if (data[ticker][i] > data["SL"][i-1]) or (data[ticker][i] < data["TP"][i-1]):
                data["Pos"][i] = 1
            if (data[ticker][i] < data["SL"][i]) or (data[ticker][i] > data["TP"][i-1]):
                data["Pos"][i] = 0
                data["Sell_Signal"][i] = 1

        elif data["Pos"][i-1] == -1:
            data["SL"][i] = data["SL"][i-1] 
            data["TP"][i] = data["TP"][i-1]
            if (data[ticker][i] < data["SL"][i-1]) or (data[ticker][i] > data["TP"][i-1]):
                data["Pos"][i] = -1
            if (data[ticker][i] > data["SL"][i]) or (data[ticker][i] < data["TP"][i-1]):
                data["Pos"][i] = 0
                data["Sell_Signal"][i] = 1

    for i in range(len(data)):
        if (data["Pos"][i] == 1): 
            data["TW"][i] = 1/len(datalist)
        elif (data["Pos"][i] == -1):
            data["TW"][i] = -(1/len(datalist))
        else:
            data["TW"][i] = 0

    chumps = data["TW"]

    return chumps

In [270]:
df = pd.DataFrame()

for i in datalist:
    temp = RSI_base(data,i)
    temp = temp.rename(i)
    df = pd.concat([df,temp],axis=1)

In [271]:
df

,NFLX,AAPL,GOOG
2015-03-20 00:00:00+00:00,-0.33333,0.00000,0.00000
2015-03-23 00:00:00+00:00,-0.33333,0.00000,0.00000
2015-03-24 00:00:00+00:00,-0.33333,0.00000,0.00000
2015-03-25 00:00:00+00:00,-0.33333,-0.33333,0.00000
2015-03-26 00:00:00+00:00,-0.33333,-0.33333,0.00000
...,...,...,...
2020-10-05 00:00:00+00:00,0.33333,0.33333,0.33333
2020-10-06 00:00:00+00:00,0.33333,0.33333,0.33333
2020-10-07 00:00:00+00:00,0.33333,0.33333,0.33333
2020-10-08 00:00:00+00:00,0.33333,0.33333,0.33333


In [247]:
import bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [286]:
data = data[['NFLX', 'AAPL', 'GOOG']]

In [291]:
crosses = bt.Strategy('Test1', [bt.algos.WeighTarget(df),
                                    bt.algos.Rebalance()])


long_only = bt.Strategy('Benchmark', [bt.algos.RunOnce(),
                        bt.algos.SelectAll(),
                        bt.algos.WeighEqually(),
                        bt.algos.Rebalance()])

t = bt.Backtest(crosses,data,commissions=lambda q, p: max(1, abs(q) * 0.05))
s = bt.Backtest(long_only, data,commissions=lambda q, p: max(1, abs(q) * 0.05))

In [292]:
report = bt.run(t,s)
report.plot()
plt.title("Portfolio Returns")
plt.savefig("arg.png")

Test1
0% [############################# ] 100% | ETA: 00:00:00Benchmark
0% [############################# ] 100% | ETA: 00:00:00

In [293]:
report.set_riskfree_rate(0.001)
report.display()

Stat                 Test1       Benchmark
-------------------  ----------  -----------
Start                2015-02-01  2015-02-01
End                  2020-10-09  2020-10-09
Risk-free rate       0.10%       0.10%

Total Return         189.47%     424.46%
Daily Sharpe         0.93        1.14
Daily Sortino        1.49        1.88
CAGR                 20.55%      33.83%
Max Drawdown         -30.97%     -34.69%
Calmar Ratio         0.66        0.98

MTD                  4.01%       5.03%
3m                   5.45%       9.00%
6m                   28.16%      48.02%
YTD                  9.41%       52.04%
1Y                   27.37%      83.19%
3Y (ann.)            26.36%      35.31%
5Y (ann.)            20.09%      31.69%
10Y (ann.)           20.55%      33.83%
Since Incep. (ann.)  20.55%      33.83%

Daily Sharpe         0.93        1.14
Daily Sortino        1.49        1.88
Daily Mean (ann.)    21.26%      33.40%
Daily Vol (ann.)     22.69%      29.17%
Daily Skew           -0.27      